In [ ]:
from threading import Thread
import json
from datetime import datetime
from time import sleep
import time
from random import random
import sys
from math import *

In [ ]:
c_partDieu = (45.76069, 4.85897)
c_zoo = (45.77818, 4.85675)
c_fourviere = (45.76247,4.82253)
c_if = (45.78170,4.87286)

v_pieton = 4
v_bouchons = 7
v_voiture = 20
v_metro = 25
v_velo = 15

In [ ]:
class phone(Thread):
    D2R = pi / 180
    
    def __init__(self, number, precision=10, t0=time.time(), delay=10, path=[0, (45.764,4.8357)], instantData = False):
        Thread.__init__(self)
        self.number = number
        self.precision = precision
        self.coordinates = path[0][1]
        self.time = t0
        self.delay = delay
        self.dead = False
        self.path= path[1:]
        self.currentTarget = self.coordinates
        self.currentSpeed = 0
        self.instantData = instantData
        pass
    
    def kill(self):
        self.dead = True
        
    def set_delay(self, delay):
        self.delay = delay
    
    def run(self):
        while(not self.dead):
            if self.currentTarget[0] == self.coordinates[0] and self.currentTarget[1] == self.coordinates[1]:
                if len(self.path)>0:
                    self.currentTarget = self.path[0][1]
                    self.currentSpeed = self.path[0][0]
                    self.path = self.path[1:]
                    dlat = (self.coordinates[0] - self.currentTarget[0]) * phone.D2R
                    dlon = (self.coordinates[1] - self.currentTarget[1]) * phone.D2R
                    a = pow(sin(dlat/2.0), 2) + cos(self.coordinates[0]*phone.D2R) * cos(self.currentTarget[0]*phone.D2R) * pow(sin(dlon/2.0), 2)
                    c = 2 * atan2(sqrt(a), sqrt(1-a))
                    d = 6367 * c
                    self.iters = ceil(d/self.currentSpeed*3600/self.delay)#Delay is in seconds
                    self.iterSize = ((self.currentTarget[0] - self.coordinates[0])/self.iters, (self.currentTarget[1] - self.coordinates[1])/self.iters)
                    
                    print("new target set, will reach in %d seconds"%(self.iters*self.delay))
                else:
                    self.iters = 0
            pdir = {}
            if self.iters >0:
                self.iters -= 1
                if self.iters == 0:
                    print("target reached")
                    self.coordinates = self.currentTarget
                else:
                    self.coordinates = tuple(map(sum,zip(self.coordinates,self.iterSize)))# Python's magic
            else:
                #todo : add something
                if (self.instantData):
                    print ("Done.")
                    break
                else: # Not needed but more readable
                    print("no more target, idling")
                    pass
            pdir['latitude']=self.coordinates[0]
            pdir['longitude']=self.coordinates[1]
            pdir['radius']=self.precision
            pdir['msisdn']=self.number
            pdir['timestamp'] = self.time
            s = json.dumps(pdir)
            print(s)
            print(sys.getsizeof(s))
            if not self.instantData:
                sleep(self.delay)
            self.time += self.delay
            
        print("%s killed"%self.number)
    

In [ ]:
phone1 = phone(12345, delay=30, path=[(0,c_fourviere),(v_pieton,c_zoo),(v_metro,c_partDieu)], instantData=True)
phone1.start()

In [ ]:
phone1.kill()

In [ ]:
etudiants = []
for i in range (1,5000):
    et =phone(i, delay = 30, path = [(0,c_if),(v_pieton,c_zoo)])
    etudiants.append(et)
    et.start()

In [ ]:
for et in etudiants:
    et.kill() # You cold blooded murderer !